In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Observational cosmology - fourth year assignment
Vanya Spasova

{This code fits low and high redshifts simultaneously,and treats CurlyM as a free parameter alongisde Omega_M, and Omega_L}
There are three different codes for each case - omega_m + omega_L = 1, omega_m + omega_L >1, and omega_m +  omega_L <1. This will
make the corresponding corner plots for each case, and makes the calculation easier, instead of running if statements in the model function
"""
#------------------------------------------------
#       omega_m +  omega_L = 1 (Flat Model)
#------------------------------------------------
from __future__ import print_function, division
from astropy.cosmology import WMAP9 as cosmo
import os
import sys
import numpy as np

# import CPNest
from cpnest.model import Model
import cpnest

In [ ]:
# import data
data = np.loadtxt('data.dat')
cosmo.H(0)     #using astropy to extract H(0)
c = 299792.458 #speed of light in vacuum in km/s

# Extract needed array from data file
z = data[:,0]           #redshift (low and high) extracted from the data
sigmaz = data[:,1]
meff  = data[:,7]       #mb_eff extracted from the data
sigmam = data[:,8]
sigma = np.mean(sigmam) #mean value of sigmam array

# Define natural log of 2pi, and sigma needed for likelihood  function
LN2PI = np.log(2.*np.pi)
LNSIGMA = np.log(sigma)

# Define the theoretical  cosmologies
def CosmologyModel(z, params): #params is a dictionary holding each of the parameter values
    # extract parameter values from the dictionary
    # To estimate only two of these parameters separately - comment out one of them & remove the bounds
    # then substitute in the integrand the corresponding definition of the parameters in terms of the relation:
    # omega_m +  omega_L = 1; e.g. omega_L = 1 - omega_m; omega_m = 1-omega_L
    CurlyM = params['CurlyM']
    omega_m = params['omega_m']
    omega_L = params['omega_L']
    #z  spacing for trapezium rule - I reduced this to speed it up
    dz = 0.01
    # calculate m(z) for each value of z from array
    result=[]                          # an array for the resulting function
    for zi in z:
        z_array = np.arange(0,zi,dz)
        #omega_L + omega_m == 1
        integrand = ((1+z_array)**2 *(1+omega_m*z_array)
        -(z_array)*(2+z_array)*omega_L)**(-0.5)
        result.append(CurlyM +5*np.log10(c*(1+zi)*np.trapz(integrand,dx=dz)))
    return result

# Set the likelihood function
class ParamEstim(Model):


    def __init__(self, names, bounds, data, modelfunc, sigma):
        # set the data
        self._data = data              # oberserved mb_eff
        self.bounds = bounds           # bounds on parameters
        self.names = names             # names of parameters
        self._sigma = sigma            # standard deviation of m_eff & z
        self._logsigma = np.log(sigma) # log sigma here to save computations in the likelihood
        self._ndata = len(self._data)  # number of data points
        self._model = modelfunc        # model function

    def log_likelihood(self, livepoint):
        """
        The natural logarithm of the likelihood function.
        """
        # calculate the model
        model = self._model(z, params=livepoint) #need to change to the correct arguments

        norm = -0.5*self._ndata*LN2PI - self._ndata*np.log(sigma)
        # chi-squared
        chisq = np.sum(((self._data - model)/(self._sigma))**2)
        print(chisq)
        return norm - 0.5*chisq
# Parameter names
names = ['CurlyM', 'omega_L', 'omega_m']

# Set bounds on the parameters
bounds = [[-3.5, -2],
          [-0.5, 1],
          [0, 1.5]] #omega_m < 0 is unphysical


# Set up the cpnest model
mod = ParamEstim(names, bounds, meff, CosmologyModel, sigma)

# Settings for the sampler
cpnest_dict = {'nlive': 1024,        # number of live points
               'nthreads': 2,        # number of parallel samplers
               'verbose': 3,         # verbosity, 0=silent, 1=progress, 2=diagnostic, 3=detailed diagnostic
               'output': "./",       # output directory
               'resume': "resume",   # determines whether cpnest will resume a run or run from scratch
               'maxmcmc': 1024       # maximum MCMC chain length
              }

# Set up the sampler
cpn = cpnest.CPNest(usermodel=mod, **cpnest_dict)

# Run the sampler
cpn.run()

# Save posterior samples
cpn.get_posterior_samples(filename='posterior.dat')
# This code produces corner plots itself, but I have preferred to use a supplementary code
# to make the plots look better
# Produce plots
cpn.plot()

2025-05-17, 09:07:18 - cpnest.cpnest.CPNest                  : Running with 1 parallel threads

CPNEST: populate samplers:   0%|                       | 0/1024 [00:00<?, ?it/s]Process Process-2:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/cpnest_env/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/cpnest_env/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/cpnest_env/lib/python3.9/site-packages/cpnest/sampler.py", line 202, in produce_sample
    self._produce_sample()
  File "/opt/anaconda3/envs/cpnest_env/lib/python3.9/site-packages/cpnest/sampler.py", line 214, in _produce_sample
    self.reset()
  File "/opt/anaconda3/envs/cpnest_env/lib/python3.9/site-packages/cpnest/sampler.py", line 116, in reset
    for n in tqdm(range(self.poolsize), desc='SMPLR {} init draw'.format(self.thread_id),
AttributeError: 'MetropolisHastin